# A propos du trait Clone

---

> Nous avons vu dans ce chapitre que les types alloués dynamiquement n'implémentent jamais le trait Copy et ne peuvent donc pas être copiées au nez et a la barbe du développeur. Afin de pouvoir être dupliqués, il est nécessaire d'utiliser la méthode clone() du trait Clone.
```
pub trait Clone: Sized {
    // Required method
    fn clone(&self) -> Self;

    // Provided method
    fn clone_from(&mut self, source: &Self) { ... }
}
```
> Les nouvelles allocation de memoire se font donc uniquement **explicitement**.

## Dérivation de Clone

- Si une structure ou une énumération contient des items alloués dynamiquement, elle ne pourra en aucun cas dériver Copy, car rappelez-vous, un Dérive d'un trait pour un type contenant des sous-types ne fonctionne que si tous les sous-types implémentent ce même trait :

In [4]:
{
    #[derive(Clone, Debug)]  // Clone Derive
    struct Remote {
        ipv4: (u8, u8, u8, u8),
        port: u32,
        server_name: String,
    }
    let r1 = Remote {
        ipv4: (192, 168, 41, 1),
        port: 8080,
        server_name: "xp6".into(),
    };
    let r2 = r1.clone();
    dbg!(r1);
    dbg!(r2);
}

[src/lib.rs:35] r1 = Remote {
    ipv4: (
        192,
        168,
        41,
        1,
    ),
    port: 8080,
    server_name: "xp6",
}
[src/lib.rs:36] r2 = Remote {
    ipv4: (
        192,
        168,
        41,
        1,
    ),
    port: 8080,
}
    server_name: "xp6",


()

## Dérive récursif

- Si une structure contient une structure qui contient une structure qui contient un type Clonable, pour pouvoir cloner ma structure de départ, il faudra que toutes les sous-structures implémentent Clone aussi :

In [12]:
{
    #[derive(Clone)]
    struct A(String);
    #[derive(Clone)]
    struct B(A);
    #[derive(Clone)]
    struct C(B);
    #[derive(Clone)]
    struct D(C);
    let d1 = D(C(B(A("banane".into()))));
    println!("{}", d1.0.0.0.0);
    
    let d2 = d1.clone(); // Possible seulement parce chaque structure derive le trait Clone
    println!("{}", d2.0.0.0.0);
}


banane
banane


()

## Implémentation obligatoire de Clone pour Copy

> Dans le cas où tous mes types sont Copy, je ne peux pas dériver seulement le trait Copy à cause d'une contrainte de trait dans l'implémentation de Copy :
```
Trait std::marker::Copy

pub trait Copy: Clone { }
```

- C'est pour cette raison que l'on croise un Dérive de Copy, il y a toujours avec lui un Dérive de Clone :

In [17]:
{
    #[derive(Copy)] // Le trait bound n'est pas satisfait !
    struct A(u32);
    dbg!(A(42).0);
}

Error: the trait bound `A: Clone` is not satisfied

In [16]:
{
    #[derive(Copy, Clone)]
    struct A(u32);
    dbg!(A(42).0);
}

[src/lib.rs:25] A(42).0 = 42


()